# **RECORDATORIO:** SUBIR "btcusd_1-min_data.csv" A 01_raw

# **IMPORTACIONES** 

In [6]:
#Ejecutar las importaciones más generales
import pandas as pd
import seaborn as sns
import numpy as np
import os, sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# **GENERACIÓN DEL DATASET FINAL** 
*Al final de los siguientes bloques de código deberías tener un dataset en 03_primary - Este considera el formato UNIX -> DIA y se encuentra escalado para el entrenamiento de modelos*

### SUBIDA INICIAL DE DATASET BASE Y CREACIÓN DE DAILY, FORMATO UNIX (MIN) -> DIA 

In [8]:
# 🔙 Subir una carpeta desde la ubicación actual del notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ruta del archivo original (minutos)
path = os.path.join(parent_dir, "data", "01_raw", "btcusd_1-min_data.csv")

print("📄 Archivo detectado en:", path)

# Verificar existencia
if not os.path.exists(path):
    raise FileNotFoundError(f"❌ No se encontró el archivo en: {path}")

# Detectar tamaño del archivo
file_size_mb = os.path.getsize(path) / (1024**2)
print(f"📦 Tamaño del archivo: {file_size_mb:.2f} MB")

# Crear acumulador de resultados
df_daily_total = []

# Leer todo el archivo por chunks (procesa el dataset completo)
chunk_size = 500_000  # medio millón de filas por bloque (~200 MB aprox)
print("🚀 Procesando dataset completo en bloques...")

for i, chunk in enumerate(pd.read_csv(path, chunksize=chunk_size)):
    print(f"🧩 Procesando bloque {i+1}...")

    # Asegurar conversión de tiempo
    chunk['Timestamp'] = pd.to_datetime(chunk['Timestamp'], unit='s')
    chunk = chunk.set_index('Timestamp')

    # Resample diario (1D)
    df_daily = chunk.resample('1D').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })
    df_daily_total.append(df_daily)

# Unir todos los bloques diarios
print("🧮 Combinando resultados...")
df_daily_full = pd.concat(df_daily_total).sort_index()

# Agrupar otra vez para consolidar (por si hay solapamiento entre chunks)
df_daily_full = df_daily_full.groupby(df_daily_full.index).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna(subset=['Open'])

# 🆕 Agregar columna 't' = número de día consecutivo
df_daily_full['t'] = range(1, len(df_daily_full) + 1)

print("✅ Transformación completa.")
print("Dimensiones finales:", df_daily_full.shape)
display(df_daily_full.head())

# Guardar versión diaria en formato Parquet (más rápido y comprimido)
output_path = os.path.join(parent_dir, "data", "02_intermediate", "btcusd_daily.parquet")
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df_daily_full.to_parquet(output_path, index=True, compression="snappy")
print(f"💾 Dataset diario guardado en formato Parquet en: {output_path}")

📄 Archivo detectado en: c:\Users\AlexandreKz\Documents\GitHub\EVAL3_IDN\data\01_raw\btcusd_1-min_data.csv
📦 Tamaño del archivo: 358.74 MB
🚀 Procesando dataset completo en bloques...
🧩 Procesando bloque 1...
🧩 Procesando bloque 2...
🧩 Procesando bloque 3...
🧩 Procesando bloque 4...
🧩 Procesando bloque 5...
🧩 Procesando bloque 6...
🧩 Procesando bloque 7...
🧩 Procesando bloque 8...
🧩 Procesando bloque 9...
🧩 Procesando bloque 10...
🧩 Procesando bloque 11...
🧩 Procesando bloque 12...
🧩 Procesando bloque 13...
🧩 Procesando bloque 14...
🧩 Procesando bloque 15...
🧮 Combinando resultados...
✅ Transformación completa.
Dimensiones finales: (4994, 6)


,Open,High,Low,Close,Volume,t
Timestamp,,,,,,
2012-01-01,4.58,4.84,4.58,4.84,10.000000,1
2012-01-02,4.84,5.00,4.84,5.00,10.100000,2
2012-01-03,5.00,5.32,5.00,5.29,107.085281,3
2012-01-04,5.29,5.57,4.93,5.57,107.233260,4
2012-01-05,5.57,6.46,5.57,6.42,70.328742,5


💾 Dataset diario guardado en formato Parquet en: c:\Users\AlexandreKz\Documents\GitHub\EVAL3_IDN\data\02_intermediate\btcusd_daily.parquet


In [9]:
# 🔙 Subir una carpeta desde la ubicación actual del notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ruta del dataset diario (Parquet)
daily_path = os.path.join(parent_dir, "data", "02_intermediate", "btcusd_daily.parquet")

print("📄 Archivo detectado en:", daily_path)

# Verificar existencia
if not os.path.exists(daily_path):
    raise FileNotFoundError(f"❌ No se encontró el archivo en: {daily_path}")

# Cargar dataset diario
btc_daily = pd.read_parquet(daily_path)

# Confirmar carga
print(f"✅ Dataset diario cargado correctamente desde:\n{daily_path}")
print(f"Dimensiones: {btc_daily.shape}")
display(btc_daily.head())
display(btc_daily.tail())

📄 Archivo detectado en: c:\Users\AlexandreKz\Documents\GitHub\EVAL3_IDN\data\02_intermediate\btcusd_daily.parquet
✅ Dataset diario cargado correctamente desde:
c:\Users\AlexandreKz\Documents\GitHub\EVAL3_IDN\data\02_intermediate\btcusd_daily.parquet
Dimensiones: (4994, 6)


,Open,High,Low,Close,Volume,t
Timestamp,,,,,,
2012-01-01,4.58,4.84,4.58,4.84,10.000000,1
2012-01-02,4.84,5.00,4.84,5.00,10.100000,2
2012-01-03,5.00,5.32,5.00,5.29,107.085281,3
2012-01-04,5.29,5.57,4.93,5.57,107.233260,4
2012-01-05,5.57,6.46,5.57,6.42,70.328742,5


,Open,High,Low,Close,Volume,t
Timestamp,,,,,,
2025-08-29,112581.0,112654.0,107488.0,108362.0,1977.324125,4990
2025-08-30,108376.0,108924.0,107389.0,108827.0,790.830131,4991
2025-08-31,108827.0,109503.0,108092.0,108269.0,747.367797,4992
2025-09-01,108268.0,109907.0,107270.0,109244.0,1584.061806,4993
2025-09-02,109255.0,111775.0,108426.0,111198.0,2117.614083,4994


### Escalado con scikitlearn

In [11]:
# Seleccionar columnas a escalar
cols = ['Low', 'High', 'Close', 'Volume']

# Inicializar escalador
scaler = StandardScaler()

# Ajustar y transformar
scaled_data = scaler.fit_transform(btc_daily[cols])

# Crear nuevo DataFrame escalado
btc_scaled = pd.DataFrame(scaled_data, columns=cols, index=btc_daily.index)

# Verificar resultados
print("📊 Primeras filas del dataset escalado:\n")
display(btc_scaled.head())

print("\n📈 Estadísticas tras el escalado (deben tener media ≈ 0 y std ≈ 1):\n")
display(btc_scaled.describe().T[['mean', 'std']].round(4))

📊 Primeras filas del dataset escalado:



,Low,High,Close,Volume
Timestamp,,,,
2012-01-01,-0.705891,-0.710982,-0.708564,-0.825010
2012-01-02,-0.705881,-0.710977,-0.708558,-0.824999
2012-01-03,-0.705875,-0.710965,-0.708548,-0.814280
2012-01-04,-0.705878,-0.710957,-0.708537,-0.814264
2012-01-05,-0.705854,-0.710925,-0.708507,-0.818342



📈 Estadísticas tras el escalado (deben tener media ≈ 0 y std ≈ 1):



,mean,std
Low,-0.0,1.0001
High,0.0,1.0001
Close,0.0,1.0001
Volume,-0.0,1.0001
